In [108]:
import os

In [109]:
from bs4 import BeautifulSoup
import cloudscraper
import geopandas as gpd

In [110]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get Winery Data

In [90]:
winery_list = []

url_root = "https://www.virginiawine.org"
scraper = cloudscraper.create_scraper()
r = scraper.get(os.path.join(url_root, "wineries/all"))
states_winery_soup = BeautifulSoup(r.text, "html.parser")
winery_lis = states_winery_soup.find_all("li", {"class": "winery-list__item"})

for winery_li in tqdm(winery_lis, desc="Looping Through wineries"):
    winery_url = f"{url_root}{winery_li.find('a').attrs['href']}"
    r_winery = scraper.get(winery_url)
    winery_soup = BeautifulSoup(r_winery.text, "html.parser")
    try:
        map_div = winery_soup.find("div", {"class": "card__map"})
        point = Point(
            float(map_div.attrs["data-lng"]), float(map_div.attrs["data-lat"])
        )
        winery_list.append(point)
    except Exception as e:
        print(e)
        print(winery_url)

Looping Through wineries:   0%|          | 0/300 [00:00<?, ?it/s]

could not convert string to float: ''
https://www.virginiawine.org/wineries/dogwood-thistle-wine-company


In [92]:
wineries_gdf = gpd.GeoDataFrame({"geometry": winery_list}, crs=4326)

In [113]:
wineries_gdf = wineries_gdf.to_crs(3968)
wineries_gdf.to_file("data/wineries.gpkg")